# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,avarua,-21.2078,-159.7750,28.03,74,40,6.17,CK,1707680073
1,1,blackmans bay,-43.0167,147.3167,14.18,86,100,1.06,AU,1707679868
2,2,margaret river,-33.9500,115.0667,17.80,78,43,7.36,AU,1707680074
3,3,vila franca do campo,37.7167,-25.4333,18.27,99,51,7.31,PT,1707680074
4,4,castillos,-34.1667,-53.8333,32.49,47,0,7.75,UY,1707680075


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
cities_hum = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    size = 'Humidity' ,
    color = 'City',
    hover_cols = ['City'],
    alpha = 0.5
)

# Display the map
cities_hum

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
criteria_df = city_data_df[(city_data_df['Max Temp'] < 30) & (city_data_df['Max Temp'] > 15) & (city_data_df['Humidity'] < 30)]

# Drop any rows with null values
my_cities = criteria_df.dropna(how='any')

# Display sample data
my_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
31,31,oum hadjer,13.2954,19.6966,28.37,10,0,4.23,TD,1707680059
32,32,ubari,26.5921,12.7805,20.81,9,48,4.04,LY,1707680089
83,83,goundam,16.4145,-3.6708,25.91,7,0,2.67,ML,1707680112
88,88,al kharijah,25.4514,30.5464,15.23,26,49,1.61,EG,1707680115
113,113,metlili chaamba,32.2667,3.6333,17.52,29,16,3.09,DZ,1707680127


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy() 

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df ['Hotel Name'] =''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
31,oum hadjer,TD,13.2954,19.6966,10,
32,ubari,LY,26.5921,12.7805,9,
83,goundam,ML,16.4145,-3.6708,7,
88,al kharijah,EG,25.4514,30.5464,26,
113,metlili chaamba,DZ,32.2667,3.6333,29,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accomodation.hotel'
limit = 1


params = {
    'categories':categories,
    'apiKey':geoapify_key,
    limit:limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = my_cities.loc[index, 'Lat']
    longitude = my_cities.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
oum hadjer - nearest hotel: No hotel found
ubari - nearest hotel: No hotel found
goundam - nearest hotel: No hotel found
al kharijah - nearest hotel: No hotel found
metlili chaamba - nearest hotel: No hotel found
mandvi - nearest hotel: No hotel found
tamanrasset - nearest hotel: No hotel found
pindiga - nearest hotel: No hotel found
ar ruseris - nearest hotel: No hotel found
khedbrahma - nearest hotel: No hotel found
bechar - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
as sulayyil - nearest hotel: No hotel found
kuraymah - nearest hotel: No hotel found
tanout - nearest hotel: No hotel found
atar - nearest hotel: No hotel found
kawm umbu - nearest hotel: No hotel found
wad rawah - nearest hotel: No hotel found
reggane - nearest hotel: No hotel found
al qitena - nearest hotel: No hotel found
waddan - nearest hotel: No hotel found
faya-largeau - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
31,oum hadjer,TD,13.2954,19.6966,10,No hotel found
32,ubari,LY,26.5921,12.7805,9,No hotel found
83,goundam,ML,16.4145,-3.6708,7,No hotel found
88,al kharijah,EG,25.4514,30.5464,26,No hotel found
113,metlili chaamba,DZ,32.2667,3.6333,29,No hotel found
129,mandvi,IN,22.8333,69.3667,21,No hotel found
165,tamanrasset,DZ,22.7850,5.5228,8,No hotel found
177,pindiga,NG,9.9833,10.9333,16,No hotel found
240,ar ruseris,SD,11.8659,34.3869,14,No hotel found
245,khedbrahma,IN,24.0333,73.0500,23,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
map_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    scale = 3,
    size = 'Humidity',
    color = 'City',
    hover_cols = ['Hotel Name', 'Country'],
    alpha = 0.5
)

# Display the map
map_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)